In [25]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import lightgbm as lgb
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [26]:
train = pd.read_csv("./final_train.csv")

In [3]:
x = train.drop("matched_score",axis=1)
y = train["matched_score"]

In [27]:
def high_corelated(data):
    correlation_matrix = data.corr().abs()
    # Select the upper triangle of the correlation matrix
    upper_triangle = correlation_matrix.where(
        np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool)
    )

    threshold = 0.8
    to_drop = [
        column for column in upper_triangle.columns if any(upper_triangle[column] > threshold)
    ]

    # Drop highly correlated features
    data_reduced = data.drop(columns=to_drop)
    return data_reduced
x_reduced = high_corelated(x)

In [5]:
from sklearn.linear_model import Lasso

model = Lasso(alpha=0.01)  # Adjust alpha for regularization strength
model.fit(x_reduced, y)
important_features = [feature for feature, coef in zip(x_reduced.columns, model.coef_) if coef != 0]
X_reduced = x_reduced[important_features]

In [28]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_reduced, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(x_train)
test_scaled = scaler.transform(x_test)

In [8]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import optuna

In [7]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.6 MB/s eta 0:00:00


In [29]:
# Convert to DMatrix for XGBoost
dtrain = xgb.DMatrix(train_scaled, label=y_train)
dval = xgb.DMatrix(test_scaled, label=y_test)

# Define the objective function for Optuna
def objective(trial):
    # Define hyperparameter search space
    param = {
        "objective": "reg:squarederror",
        "eval_metric": "rmse",
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.2),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "min_child_weight": trial.suggest_loguniform("min_child_weight", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "n_estimators": trial.suggest_int("n_estimators", 100, 500)
    }

    # Train the model
    model = xgb.train(param, dtrain, evals=[(dval, "val")], early_stopping_rounds=50, verbose_eval=False)

    # Predict on the validation set
    y_val_pred = model.predict(dval)

    # Calculate MSE
    mse = mean_squared_error(y_test, y_val_pred)
    return mse

# Create an Optuna study
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=1000)

# Get the best parameters and best score
best_params = study.best_params
best_score = study.best_value

print("Best Parameters:", best_params)
print("Best MSE:", best_score)

[I 2024-12-27 18:56:14,293] A new study created in memory with name: no-name-c7b4533f-469c-44c6-a76f-2ed8280cec6e
[I 2024-12-27 18:56:14,381] Trial 0 finished with value: 0.017512090938468932 and parameters: {'learning_rate': 0.08339931286007128, 'max_depth': 6, 'min_child_weight': 4.322700951583105, 'subsample': 0.9647709807684529, 'colsample_bytree': 0.602131238759303, 'n_estimators': 199}. Best is trial 0 with value: 0.017512090938468932.
[I 2024-12-27 18:56:14,540] Trial 1 finished with value: 0.023782828272652427 and parameters: {'learning_rate': 0.011982176500897659, 'max_depth': 10, 'min_child_weight': 2.3420820678761523, 'subsample': 0.6709863578447793, 'colsample_bytree': 0.7504086428825512, 'n_estimators': 172}. Best is trial 0 with value: 0.017512090938468932.
[I 2024-12-27 18:56:14,720] Trial 2 finished with value: 0.022126651061729858 and parameters: {'learning_rate': 0.021097282244747266, 'max_depth': 9, 'min_child_weight': 7.365888615062058, 'subsample': 0.86317327667171

Best Parameters: {'learning_rate': 0.1998718338559132, 'max_depth': 10, 'min_child_weight': 1.6054571642004372, 'subsample': 0.9868750929618859, 'colsample_bytree': 0.6562444110960489, 'n_estimators': 320}
Best MSE: 0.011481091425232922


In [9]:
!pip install catboost lightgbm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.3 MB/s eta 0:00:00


In [30]:
def objective(trial):
    # Define hyperparameter search space
    param = {
        "iterations": trial.suggest_int("iterations", 100, 500),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.2),
        "depth": trial.suggest_int("depth", 4, 10),
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-3, 10),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
        "random_strength": trial.suggest_float("random_strength", 0.0, 10.0),
        "loss_function": "RMSE",
        "verbose": 0,
        "random_state": 42,
    }

    # Train the model with the current hyperparameters
    model = CatBoostRegressor(**param)
    model.fit(train_scaled, y_train, eval_set=(test_scaled, y_test), early_stopping_rounds=50, verbose=False)

    # Predict on the validation set
    y_val_pred = model.predict(test_scaled)

    # Calculate MSE
    mse = mean_squared_error(y_test, y_val_pred)
    return mse

# Create an Optuna study
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

# Get the best parameters and best score
best_params = study.best_params
best_score = study.best_value

print("Best Parameters:", best_params)
print("Best MSE:", best_score)

[I 2024-12-27 18:59:13,901] A new study created in memory with name: no-name-18907956-6c5c-4c48-8684-8f46b035dcbd
[I 2024-12-27 18:59:14,905] Trial 0 finished with value: 0.012530438083652153 and parameters: {'iterations': 325, 'learning_rate': 0.04684410738580754, 'depth': 4, 'l2_leaf_reg': 0.06262949611156342, 'bagging_temperature': 0.12013528842481302, 'random_strength': 3.6563065193211273}. Best is trial 0 with value: 0.012530438083652153.
[I 2024-12-27 18:59:17,565] Trial 1 finished with value: 0.009112485091075281 and parameters: {'iterations': 269, 'learning_rate': 0.18781725673095426, 'depth': 8, 'l2_leaf_reg': 0.24019561722125304, 'bagging_temperature': 0.7765700967839088, 'random_strength': 6.024768519417646}. Best is trial 1 with value: 0.009112485091075281.
[I 2024-12-27 18:59:27,277] Trial 2 finished with value: 0.012305966633688818 and parameters: {'iterations': 493, 'learning_rate': 0.011100732848725738, 'depth': 9, 'l2_leaf_reg': 0.08504767949570158, 'bagging_temperatur

Best Parameters: {'iterations': 421, 'learning_rate': 0.07429073841894898, 'depth': 10, 'l2_leaf_reg': 0.049015842550303744, 'bagging_temperature': 0.897895834286141, 'random_strength': 0.4920066673968035}
Best MSE: 0.008693803005061668


In [ ]:
# {'iterations': 421, 'learning_rate': 0.07429073841894898, 'depth': 10, 'l2_leaf_reg': 0.049015842550303744, 'bagging_temperature': 0.897895834286141, 'random_strength': 0.4920066673968035}

In [14]:
from catboost import CatBoostRegressor, Pool
from lightgbm import LGBMRegressor

In [ ]:
train_scaled_pred = scaler.fit_transform(X_reduced)
test_scaled_pred = scaler.transform(x_test)

In [ ]:
def objective(trial):
    # Define hyperparameter search space
    params = {
       "objective": "regression",
        "metric": "rmse",
        "boosting_type": "gbdt",
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.2),
        "num_leaves": trial.suggest_int("num_leaves", 20, 150),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 50),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 10),
        "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-3, 10),
        "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-3, 10),
        "verbosity": -1,
        "seed": 42,
    }
    trn_data = lgb.Dataset(train_scaled, y_train)
    val_data = lgb.Dataset(test_scaled, y_test)
    # Train the model with the current hyperparameters
    model = lgb.train(params,
                trn_data,
                20000,
                valid_sets=[trn_data, val_data],
                callbacks=[lgb.early_stopping(stopping_rounds=50), lgb.log_evaluation(50)])
    # model.fit(
    #     train_scaled, y_train,
    #     eval_set=[(test_scaled, y_test)],
    #     eval_metric="rmse",
    #     # early_stopping_rounds=50,
    #     verbosity=1
    # )

    # Predict on the validation set
    y_val_pred = model.predict(test_scaled)

    # Calculate MSE
    mse = mean_squared_error(y_test, y_val_pred)
    return mse

# Create an Optuna study
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=500)

# Get the best parameters and best score
best_params = study.best_params
best_score = study.best_value

print("Best Parameters:", best_params)
print("Best MSE:", best_score)

[I 2024-12-27 19:04:44,930] A new study created in memory with name: no-name-10c5d1ef-1e1a-4367-9726-e2ba1aba99fe


Training until validation scores don't improve for 50 rounds
[50]	training's rmse: 0.133773	valid_1's rmse: 0.133823
[100]	training's rmse: 0.122425	valid_1's rmse: 0.124743
[150]	training's rmse: 0.11646	valid_1's rmse: 0.120067
[200]	training's rmse: 0.112302	valid_1's rmse: 0.1167
[250]	training's rmse: 0.108883	valid_1's rmse: 0.113953
[300]	training's rmse: 0.106318	valid_1's rmse: 0.111871
[350]	training's rmse: 0.103767	valid_1's rmse: 0.109856
[400]	training's rmse: 0.101782	valid_1's rmse: 0.108371
[450]	training's rmse: 0.100107	valid_1's rmse: 0.107108
[500]	training's rmse: 0.0984412	valid_1's rmse: 0.105941
[550]	training's rmse: 0.0970836	valid_1's rmse: 0.105035
[600]	training's rmse: 0.0958108	valid_1's rmse: 0.10427
[650]	training's rmse: 0.0947384	valid_1's rmse: 0.103608
[700]	training's rmse: 0.0935622	valid_1's rmse: 0.102893
[750]	training's rmse: 0.0925751	valid_1's rmse: 0.102252
[800]	training's rmse: 0.0916304	valid_1's rmse: 0.101714
[850]	training's rmse: 0.

[I 2024-12-27 19:04:48,496] Trial 0 finished with value: 0.009173525230041439 and parameters: {'learning_rate': 0.020527452752225096, 'num_leaves': 44, 'max_depth': 5, 'min_data_in_leaf': 44, 'feature_fraction': 0.7749454445553567, 'bagging_fraction': 0.7940747022693728, 'bagging_freq': 1, 'lambda_l1': 0.3610869211855437, 'lambda_l2': 0.018345672312033136}. Best is trial 0 with value: 0.009173525230041439.


Training until validation scores don't improve for 50 rounds
[50]	training's rmse: 0.105128	valid_1's rmse: 0.111601
[100]	training's rmse: 0.0949294	valid_1's rmse: 0.10461
[150]	training's rmse: 0.088894	valid_1's rmse: 0.100793
[200]	training's rmse: 0.0846054	valid_1's rmse: 0.0986492
[250]	training's rmse: 0.081297	valid_1's rmse: 0.0977023


[I 2024-12-27 19:04:49,123] Trial 1 finished with value: 0.009394467133805248 and parameters: {'learning_rate': 0.08907166569151471, 'num_leaves': 63, 'max_depth': 6, 'min_data_in_leaf': 39, 'feature_fraction': 0.9033105974618403, 'bagging_fraction': 0.7303434365683467, 'bagging_freq': 4, 'lambda_l1': 0.001320851318840217, 'lambda_l2': 0.016177661625471233}. Best is trial 0 with value: 0.009173525230041439.


...

Training until validation scores don't improve for 50 rounds
[50]	training's rmse: 0.119971	valid_1's rmse: 0.125364
[100]	training's rmse: 0.0990618	valid_1's rmse: 0.111961
[150]	training's rmse: 0.0872241	valid_1's rmse: 0.104881
[200]	training's rmse: 0.0791911	valid_1's rmse: 0.100561
[250]	training's rmse: 0.0731679	valid_1's rmse: 0.0977764
[300]	training's rmse: 0.0685137	valid_1's rmse: 0.0959037
[350]	training's rmse: 0.0650909	valid_1's rmse: 0.0949368
[400]	training's rmse: 0.062045	valid_1's rmse: 0.094263
[450]	training's rmse: 0.0592747	valid_1's rmse: 0.0937782
[500]	training's rmse: 0.0568792	valid_1's rmse: 0.0934241
[550]	training's rmse: 0.0545675	valid_1's rmse: 0.0931636
[600]	training's rmse: 0.0525916	valid_1's rmse: 0.0930137
[650]	training's rmse: 0.0506104	valid_1's rmse: 0.0929478
[700]	training's rmse: 0.0489186	valid_1's rmse: 0.0927237
[750]	training's rmse: 0.0471642	valid_1's rmse: 0.0926358
[800]	training's rmse: 0.0456802	valid_1's rmse: 0.0925364
[85

[I 2024-12-27 19:25:07,461] Trial 343 finished with value: 0.008535272307678109 and parameters: {'learning_rate': 0.01707060887365192, 'num_leaves': 108, 'max_depth': 14, 'min_data_in_leaf': 12, 'feature_fraction': 0.6187392329131117, 'bagging_fraction': 0.8897045092781551, 'bagging_freq': 10, 'lambda_l1': 0.0070183792398152475, 'lambda_l2': 0.03865429291335428}. Best is trial 314 with value: 0.008410523373739414.


Training until validation scores don't improve for 50 rounds
[50]	training's rmse: 0.122067	valid_1's rmse: 0.12697
[100]	training's rmse: 0.101305	valid_1's rmse: 0.114465
[150]	training's rmse: 0.0885979	valid_1's rmse: 0.107626
[200]	training's rmse: 0.0803228	valid_1's rmse: 0.103872
[250]	training's rmse: 0.073932	valid_1's rmse: 0.100426
[300]	training's rmse: 0.0687304	valid_1's rmse: 0.0983721
[350]	training's rmse: 0.0647113	valid_1's rmse: 0.0972799
[400]	training's rmse: 0.0615363	valid_1's rmse: 0.0965175
[450]	training's rmse: 0.0588073	valid_1's rmse: 0.0962806
[500]	training's rmse: 0.0564493	valid_1's rmse: 0.0959048
[550]	training's rmse: 0.0540719	valid_1's rmse: 0.0956743
[600]	training's rmse: 0.0519855	valid_1's rmse: 0.0954644
[650]	training's rmse: 0.049923	valid_1's rmse: 0.0953395
[700]	training's rmse: 0.0482368	valid_1's rmse: 0.0953397
[750]	training's rmse: 0.0464895	valid_1's rmse: 0.0951895
[800]	training's rmse: 0.0450796	valid_1's rmse: 0.0951973
Early 

[I 2024-12-27 19:25:11,025] Trial 344 finished with value: 0.009047569198697427 and parameters: {'learning_rate': 0.015143518701941644, 'num_leaves': 119, 'max_depth': 14, 'min_data_in_leaf': 13, 'feature_fraction': 0.9990714914663025, 'bagging_fraction': 0.9070743421251712, 'bagging_freq': 10, 'lambda_l1': 0.008016082077960776, 'lambda_l2': 0.029575696906774713}. Best is trial 314 with value: 0.008410523373739414.


Training until validation scores don't improve for 50 rounds
[50]	training's rmse: 0.126386	valid_1's rmse: 0.129992
[100]	training's rmse: 0.10563	valid_1's rmse: 0.116206
[150]	training's rmse: 0.0936165	valid_1's rmse: 0.109024
[200]	training's rmse: 0.084748	valid_1's rmse: 0.10401
[250]	training's rmse: 0.0785187	valid_1's rmse: 0.10085
[300]	training's rmse: 0.0733594	valid_1's rmse: 0.0982278
[350]	training's rmse: 0.0695288	valid_1's rmse: 0.096638
[400]	training's rmse: 0.0662721	valid_1's rmse: 0.0956716
[450]	training's rmse: 0.0634422	valid_1's rmse: 0.0949727
[500]	training's rmse: 0.0610714	valid_1's rmse: 0.0944462
[550]	training's rmse: 0.0587671	valid_1's rmse: 0.0939397
[600]	training's rmse: 0.0568285	valid_1's rmse: 0.0937168
[650]	training's rmse: 0.0549234	valid_1's rmse: 0.0935467
[700]	training's rmse: 0.0532622	valid_1's rmse: 0.09318
[750]	training's rmse: 0.051616	valid_1's rmse: 0.0930187
[800]	training's rmse: 0.0502061	valid_1's rmse: 0.0928462
[850]	train

In [15]:
from lightgbm import LGBMRegressor

In [18]:
test_data = pd.read_csv("./final_test.csv")
sample_submission = pd.read_csv("sample_submission.csv")

In [19]:
test_data = test_data[X_reduced.columns]

In [22]:
train_scaled_pred = scaler.fit_transform(X_reduced)
test_scaled_pred = scaler.transform(test_data)

In [23]:
params = {'learning_rate': 0.05239001005705923, 'num_leaves': 52, 'max_depth': 15, 'min_data_in_leaf': 10, 'feature_fraction': 0.6347799443597311, 'bagging_fraction': 0.9927001351600075, 'bagging_freq': 2, 'lambda_l1': 0.5271067496620728, 'lambda_l2': 4.273896603768413}
model = LGBMRegressor(
  **params
)

model.fit(
    train_scaled, y_train,
    eval_set=[(test_scaled, y_test)],
    eval_metric="rmse",
)


LGBMRegressor(bagging_fraction=0.9927001351600075, bagging_freq=2,
              feature_fraction=0.6347799443597311, lambda_l1=0.5271067496620728,
              lambda_l2=4.273896603768413, learning_rate=0.05239001005705923,
              max_depth=15, min_data_in_leaf=10, num_leaves=52)

In [11]:
#prepare test file

In [24]:

test_predictions = model.predict(test_scaled_pred)

sample_submission = pd.read_csv("sample_submission.csv")

# Create the submission file
submission = sample_submission.copy()
submission["matched_score"] = test_predictions
submission.to_csv("submission.csv", index=False)

print("Submission file created: submission.csv")

Submission file created: submission.csv
